### Loading libraries

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

### Loading the model

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# The name of the models to upload
models = [
    "lightblue/suzume-llama-3-8B-multilingual-orpo-borda-half",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/Qwen2-7B-Instruct-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
] # Try more models at https://huggingface.co/models
n=2
model_name = models[n].split('/')[1]
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models[n], # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/713 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
lightblue/suzume-llama-3-8B-multilingual-orpo-borda-half does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


### Model Modification Parameters LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 1,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Preparing a dataset for training

### Post Generation

In [ ]:
import pandas as pd
import numpy as np

posts = pd.read_excel('/content/post_gen_train.xlsx')
# Replacing NaN values with empty strings
posts = posts.fillna('')

# Converting all columns to a string type
posts['Text'] = posts['Текст'].astype(str)
posts['Instruction'] = posts['Instruction'].astype(str)
posts['sys_promt'] = posts['sys_promt'].astype(str)

train = posts
train.head(3)

from datasets import Dataset
# Создание Dataset
ds = Dataset.from_dict({
    "output": posts['Text'],
    "input": posts['Instruction'],
    "instruction": posts['sys_promt']
})

ds[0]

### Converting a dataset to a suitable format

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = ds
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/2626 [00:00<?, ? examples/s]

## Model training
Now we will use the `SFTTrainer` from Huggingface TRL! More documentation is here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer ). We do 60 steps for speed, but we can set `num_train_epochs=1` for a full run and disable `max_steps=None'. We also support the `DPOTrainer` from TRL!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/2626 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Indicators of available video memory
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.594 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,626 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
5,2.590400
10,1.733800
15,1.309400
20,1.162900
25,1.154400
30,1.094800
35,0.996300
40,1.092400
45,1.025600
50,1.211700


In [ ]:
#@title Video memory usage indicators and statistics
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

613.6526 seconds used for training.
10.23 minutes used for training.
Peak reserved memory = 8.668 GB.
Peak reserved memory for training = 3.074 GB.
Peak reserved memory % of max memory = 58.774 %.
Peak reserved memory for training % of max memory = 20.844 %.



## Запуск модели

### Generating posts

In [ ]:
import re
post_result=''
for Instruction in posts["Instruction"].unique():
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            posts["sys_promt"][0], # instruction
            Instruction, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
    message_answer = tokenizer.batch_decode(outputs)[0].rsplit('Response:', 1)[-1]
    match = re.search(r'[012345]', str(Instruction))
    post_result+=str(int(match.group()))+' - '+str(message_answer)+'\n'
print(post_result)

In [ ]:

post_result=''
for Instruction in posts["Instruction"].unique():
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Answer in English\n "+str(posts["sys_promt"][0]), # instruction
            "Answer in English\n "+Instruction, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
    message_answer = tokenizer.batch_decode(outputs)[0].rsplit('Response:', 1)[-1]
    match = re.search(r'[012345]', str(Instruction))
    post_result+=str(int(match.group()))+' - '+str(message_answer)+'\n'
print(post_result)

In [ ]:

my_file = open(f"{model_name}_post_gen.txt", "w+")
my_file.write(post_result)
my_file.close()